<a href="https://colab.research.google.com/github/nrimsky/qa/blob/main/codebase_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -U InstructorEmbedding sentence-transformers pylatexenc faiss-cpu langchain openai

In [2]:
import os
import requests
import shutil
import tarfile
import re
from InstructorEmbedding import INSTRUCTOR
from pylatexenc.latex2text import LatexNodes2Text
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import torch
import os
import requests
import os
import base64
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import drive


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = input("Paste OpenAI API Key: ")

In [5]:
# Example URL: https://github.com/luc-github/ESP3D-WEBUI
repo_path = "luc-github/ESP3D-WEBUI"

In [6]:
def get_source_from_github(repo_path):
    api_url = f'https://api.github.com/repos/{repo_path}/contents'
    drive_path = '/content/drive/My Drive/'
    repo_dir = os.path.join(drive_path, repo_path.replace('/', '_')) # replacing '/' with '_' as '/' is not allowed in directory names
    file_contents = {}
    total_size = 0
    if not os.path.exists(repo_dir) or not os.listdir(repo_dir): # if the directory does not exist or is empty
        file_contents, total_size = download_from_github(api_url, repo_dir)
    else:
        for dirpath, dirnames, filenames in os.walk(repo_dir):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                try:
                    with open(filepath, 'r') as f:
                        file_content = f.read()
                        file_contents[filepath] = file_content
                        total_size += len(file_content)
                except UnicodeDecodeError:
                    pass
    if total_size <= 1000:
        print(f"Failed to extract enough source data - file content size = {total_size} chars")
    return file_contents

def download_from_github(api_url, local_dir):
    file_contents = {}
    total_size = 0
    response = requests.get(api_url)
    if response.status_code == 200:
        files = response.json()
        for file in files:
            if file['type'] == 'file':
                file_response = requests.get(file['download_url'])
                if file_response.status_code == 200:
                    local_file_path = os.path.join(local_dir, file['path'])
                    os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
                    with open(local_file_path, 'wb') as f:
                        f.write(file_response.content)
                    try:
                        with open(local_file_path, 'r') as f:
                            file_content = f.read()
                            file_contents[local_file_path] = file_content
                            total_size += len(file_content)
                    except UnicodeDecodeError:
                        continue
            elif file['type'] == 'dir':
                sub_dir_contents, sub_dir_size = download_from_github(file['url'], local_dir)
                file_contents.update(sub_dir_contents)
                total_size += sub_dir_size
    else:
        print(f'Error: received status code {response.status_code} from GitHub.')
    return file_contents, total_size

In [7]:
def extract_all_text_chunks(repo_path, chunk_size = 1000):
    text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap  = 0,
      length_function = len,
    )
    all_files = get_source_from_github(repo_path)
    documents = text_splitter.create_documents(texts = list(all_files.values()), metadatas = [{"source": p} for p in list(all_files.keys())])
    return documents


In [8]:
chunks = extract_all_text_chunks(repo_path)

In [9]:
chunks[10]

Document(page_content="## What's Changed\n\n# Fixes\n* Fix wrong source show in file panel\n* Fix SSID with ' not properly transfered from dialog box  to control\n* Fix macro panel overload UI on some resolutions\n* Fix moving buttons in grbl\n* Fix SD icon and create dir displayed on GRBL thanks  @Wesie\n* Fix Abort button for marlin-embedded and marlinkimbra thanks @GerogeFu\n* Fix always need to save preferences even no change\n* Fix display glitch\n* Fix Repetier ok 0 still visible with verbose mode off\n* Fix Setup not working anymore due to breaking changes in GRBL_ESP32\n* Fix T is not T0 when heating T1 and so display wrong value thanks @Drzet\n* Fix typo thanks @terjeio\n* Fix bug with pure GRBL no receiving commands feedback thanks @jjhamb\n* Fix pur GRBL not managing positions properly\n* Fix some typo - thanks @kondorzs\n* Fix smoothieware cannot list sub directory - thanks @sns5400\n* Fix missing translation for SD", metadata={'source': '/content/drive/My Drive/luc-github_

In [10]:
len(chunks)

2471

In [11]:
model = INSTRUCTOR('hkunlp/instructor-xl')

load INSTRUCTOR_Transformer
max_seq_length  512


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Using device", device)

INDEX_TEXT = "Represent this section of a GitHub codebase for retrieval given a question about the codebase:"
RETRIEVAL_TEXT = "Represent this question about a codebase for retrieving relevant code snippets:"

def encode_instructor(instruction, sentences):
    return model.encode([[instruction,sentence] for sentence in sentences])

class InstructorEmbeddings(Embeddings):

    def embed_documents(self, texts):
        return encode_instructor(INDEX_TEXT, texts)

    def embed_query(self, text):
        return encode_instructor(RETRIEVAL_TEXT, [text])[0]

def cli_ask_questions(repo_path):
    chunks = extract_all_text_chunks(repo_path)
    embeddings = InstructorEmbeddings()
    vectorstore = FAISS.from_documents(chunks, embeddings)
    chain_type_kwargs = {
        "prompt": ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "You are a helpful assistant that answers questions about a codebase given some snippets from the codebase. Whenever useful and possible, you directly quote the code."
            ),
            HumanMessagePromptTemplate.from_template("""
                Some relevant code snippets:

                {context}

                Question: {question}
                Answer:
            """)
        ]),
        "document_variable_name": "context"
    }

    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm=ChatOpenAI(model_name='gpt-3.5-turbo'),
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs
    )

    while True:
        question = input("Enter your question about the codebase (or 'quit' to stop): ")
        if question.lower() == 'quit':
            break
        else:
            try:
                response = qa(question)
                print(response['answer'])
                print(response['sources'])
            except Exception as e:
                print("An error occurred while processing your question.")
                print(str(e))


Using device cuda


In [ ]:
repo_path = input("Enter the relative path of the GitHub repo you want to ask questions about: ")
cli_ask_questions(repo_path)

Enter the relative path of the GitHub repo you want to ask questions about: luc-github/ESP3D-WEBUI
Enter your question about the codebase (or 'quit' to stop): How does the code configure different settings depending on the firmware type?
The code uses switch statements and conditional statements to determine the type of firmware and configure settings accordingly. For example, in the `update_UI_setting()` function, the `target_firmware` variable is set based on the value of `setting_configList[i].defaultvalue`, and then `update_UI_firmware_target()` and `init_files_panel(false)` are called based on the value of `setting_configList[i].pos`. Similarly, in the `get_config_command()` function, the `target_firmware` variable is checked to determine the appropriate command to generate based on the firmware type.

Enter your question about the codebase (or 'quit' to stop): How are the tabs switched between?
The tabs are switched between using JavaScript functions. In one case, the function is